In [2]:
import requests
import random
import time
import os
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urlparse, parse_qs
from datetime import datetime, timedelta

In [3]:
import ctypes

# SetThreadExecutionState: 시스템이 슬립하거나 화면이 꺼지는 것 방지
ctypes.windll.kernel32.SetThreadExecutionState(0x80000002)

-2147483648

In [ ]:
# KOSPI 종목 불러오기

kospi_df = pd.read_csv("kospi_list.csv", encoding="cp949")
kospi_stocks = list(set(kospi_df["종목명"].str.strip()))

print(kospi_stocks)

['KT', 'LX홀딩스', '제일약품', '한솔PNS', '대동', '대한제강', '한솔제지', '동원금속', '대상홀딩스', '테이팩스', '화천기계', '현대리바트', '삼원강재', '지누스', '현대백화점', '동국씨엠', 'LG이노텍', '웰바이오텍', '오뚜기', '일동홀딩스', '대창단조', '화승알앤에이', '동일제강', '금호건설', '현대위아', '에쓰씨엔지니어링', '덴티움', 'NH프라임리츠', '팜젠사이언스', '제이준코스메틱', 'NI스틸', '동양철관', 'DB손해보험', 'NAVER', '태경비케이', '카카오페이', '우성', '신성통상', '동양', 'DB하이텍', '지엠비코리아', 'DB', '인지컨트롤스', 'BGF', '이수화학', '세아제강', 'HJ중공업', '자이에스앤디', '한화손해보험', '인천도시가스', '삼성생명', '제주항공', '에이리츠', '계룡건설', '까뮤이앤씨', '대한화섬', '동원수산', '한진칼', 'HS애드', '삼성공조', '다스코', 'SB성보', '만호제강', 'SG세계물산', '쿠쿠홀딩스', '무학', '신송홀딩스', 'DSR', '화천기공', '고려제강', '삼아알미늄', 'SKC', '대한제당', '한국수출포장', '세이브존I&C', '에어부산', '일신석재', '아남전자', '신세계', '한컴라이프케어', '한신기계', '대덕전자', '인바이오젠', '코스맥스비티아이', '대림통상', '세아특수강', '아모레퍼시픽', '롯데관광개발', '미래산업', '동성제약', '한화갤러리아', '주연테크', 'SJM', '조선선재', '진양화학', '태원물산', 'IHQ', '롯데렌탈', '미래에셋생명', '유니온', '효성티앤씨', 'CS홀딩스', '키움증권', '삼양사', '비에이치', '한화오션', '신원', '신도리코', '율촌화학', '스틱인베스트먼트', '산일전기', '미창석유', '화승코퍼레이션', '유진투자증권', '삼일씨엔에스', '엔씨소프트', '평화홀딩스', '한국화장품',

In [21]:
# 일별 리포트 조회수 상위 n개 & 코스피 종목 크롤링 함수

base_url = "https://finance.naver.com"

def get_report_list(date, page=1):
    params = {
        "searchType": "writeDate",
        "writeFromDate": date,
        "writeToDate": date,
        "page": page,
    }

    headers = {"User-Agent": "Mozilla/5.0"}

    res = requests.get(
        f"{base_url}/research/company_list.naver", params=params, headers=headers
    )
    soup = BeautifulSoup(res.text, "html.parser")

    last_page = (
        int(soup.select_one(".pgRR a")["href"].split("page=")[-1])
        if soup.select_one(".pgRR a")
        else 1
    )

    data = []
    table = soup.select("table.type_1")[0]
    for row in table.select("tr")[2:-3]:
        cols = row.select("td")
        if len(cols) != 6:
            continue
        item = {
            "종목명": cols[0].text.strip(),
            "제목": cols[1].text.strip(),
            "증권사": cols[2].text.strip(),
            "작성일": cols[4].text.strip(),
            "조회수": cols[5].text.strip(),
            "URL": f"{base_url}/research/{cols[1].select_one('a')['href']}",
        }
        data.append(item)

    return data, last_page


def crawl_reports_by_date(date):
    all_data = []
    page_data, total_pages = get_report_list(date, 1)
    all_data.extend(page_data)

    for page in range(2, total_pages + 1):
        page_data, _ = get_report_list(date, page)
        all_data.extend(page_data)
        time.sleep(0.3)

    kospi_data = [item for item in all_data if item["종목명"] in kospi_stocks]

    # DataFrame 생성 및 조회수 전처리
    # df = pd.DataFrame(kospi_data)
    # if not df.empty:
    #   df["조회수"] = (
    #        df["조회수"].str.replace(",", "").astype(int)
    #    )  # 콤마 제거 후 정수 변환
    #    df = df.sort_values("조회수", ascending=False).head(30)  # 조회수 상위 10개 추출

    df = pd.DataFrame(all_data)

    # print(
    #     f"[{date}] 전체: {len(all_data)}개 / KOSPI: {len(kospi_data)}개 → 최종 저장: {len(df)}개"
    # )
    print(f"[{date}] 최종 저장: {len(df)}개")

    return df


def crawl_range(start_date, end_date):
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")

    full_df = pd.DataFrame()

    while start <= end:
        date_str = start.strftime("%Y-%m-%d")
        df = crawl_reports_by_date(date_str)
        if not df.empty:
            full_df = pd.concat([full_df, df], ignore_index=True)
        start += timedelta(days=1)

    full_df.to_csv(
        f"KOSPI_reports_{start_date}_to_{end_date}.csv",
        index=False,
        encoding="utf-8-sig",
    )

In [24]:
crawl_range("2024-04-23", "2025-05-23")

[2024-04-23] 최종 저장: 32개
[2024-04-24] 최종 저장: 24개
[2024-04-25] 최종 저장: 62개
[2024-04-26] 최종 저장: 121개
[2024-04-27] 최종 저장: 0개
[2024-04-28] 최종 저장: 0개
[2024-04-29] 최종 저장: 124개
[2024-04-30] 최종 저장: 78개
[2024-05-01] 최종 저장: 0개
[2024-05-02] 최종 저장: 119개
[2024-05-03] 최종 저장: 82개
[2024-05-04] 최종 저장: 0개
[2024-05-05] 최종 저장: 0개
[2024-05-06] 최종 저장: 0개
[2024-05-07] 최종 저장: 68개
[2024-05-08] 최종 저장: 38개
[2024-05-09] 최종 저장: 75개
[2024-05-10] 최종 저장: 124개
[2024-05-11] 최종 저장: 0개
[2024-05-12] 최종 저장: 0개
[2024-05-13] 최종 저장: 88개
[2024-05-14] 최종 저장: 46개
[2024-05-15] 최종 저장: 0개
[2024-05-16] 최종 저장: 85개
[2024-05-17] 최종 저장: 68개
[2024-05-18] 최종 저장: 0개
[2024-05-19] 최종 저장: 0개
[2024-05-20] 최종 저장: 32개
[2024-05-21] 최종 저장: 27개
[2024-05-22] 최종 저장: 20개
[2024-05-23] 최종 저장: 20개
[2024-05-24] 최종 저장: 25개
[2024-05-25] 최종 저장: 0개
[2024-05-26] 최종 저장: 0개
[2024-05-27] 최종 저장: 21개
[2024-05-28] 최종 저장: 39개
[2024-05-29] 최종 저장: 13개
[2024-05-30] 최종 저장: 28개
[2024-05-31] 최종 저장: 29개
[2024-06-01] 최종 저장: 0개
[2024-06-02] 최종 저장: 0개
[2024-06-03] 최종 저장: 24개
[20

In [25]:
df = pd.read_csv("KOSPI_reports_2024-04-23_to_2025-05-23.csv")
df

,종목명,제목,증권사,작성일,조회수,URL
0,HL만도,1Q24 Preview: 하반기 모멘텀을 기대,교보증권,24.04.23,1913,https://finance.naver.com/research/company_rea...
1,현대글로비스,1Q24 Preview: 실적 회복 중,교보증권,24.04.23,1363,https://finance.naver.com/research/company_rea...
2,현대오토에버,1Q24 Preview: 계속 성장중인 점에 주목,교보증권,24.04.23,2809,https://finance.naver.com/research/company_rea...
3,현대모비스,1Q24 Preview: 상반기는 수익성 회복 구간,교보증권,24.04.23,1587,https://finance.naver.com/research/company_rea...
4,기아,1Q24 Preview: 달라진 이익 체력 증명할 것,교보증권,24.04.23,2345,https://finance.naver.com/research/company_rea...
...,...,...,...,...,...,...
9990,SPC삼립,"반복되는 안전사고, 투심 회복 요원",IBK투자증권,25.05.23,335,https://finance.naver.com/research/company_rea...
9991,인바디,인바디는 의료기기라는 사실을 잊지말자,미래에셋증권,25.05.23,494,https://finance.naver.com/research/company_rea...
9992,롯데지주,하반기 개선 가능성,IBK투자증권,25.05.23,393,https://finance.naver.com/research/company_rea...
9993,메디젠휴먼케어,유전체 분석 기반의 정밀의료 전문기업,NICE평가정보,25.05.23,274,https://finance.naver.com/research/company_rea...


In [34]:
import random

USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/125.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 Safari/605.1.15",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 Chrome/124.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 Chrome/122.0.0.0 Safari/537.36",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/109.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_2) Gecko/20100101 Firefox/110.0",
    "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:110.0) Gecko/20100101 Firefox/110.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/110.0.0.0 Safari/537.36 Edg/110.0.1587.57",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 16_2 like Mac OS X) AppleWebKit/605.1.15 Version/16.2 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Linux; Android 13; SM-S918N) AppleWebKit/537.36 Chrome/113.0.0.0 Mobile Safari/537.36",
    "Mozilla/5.0 (Linux; Android 10; SM-G970F) AppleWebKit/537.36 Chrome/80.0.3987.119 SamsungBrowser/13.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/124.0.0.0 Safari/537.36 Brave/124.0.0.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/123.0.0.0 Safari/537.36 OPR/89.0.4447.83",
    "Mozilla/5.0 (X11; Linux x86_64) Chrome/117.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; WOW64) Chrome/118.0.5993.90 Safari/537.36",
    "Mozilla/5.0 (iPad; CPU OS 15_5 like Mac OS X) AppleWebKit/605.1.15 Version/15.5 Mobile/15E148 Safari/604.1",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/125.0.0.1 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) Gecko/20100101 Firefox/111.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 11_6_5) AppleWebKit/605.1.15 Version/15.5 Safari/605.1.15",
    "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 Chrome/104.0.0.0 Safari/537.36",
]


def get_random_headers():
    return {
        "User-Agent": random.choice(USER_AGENTS),
        "Referer": "https://finance.naver.com/",
    }

In [50]:
def get_report_content(url):
    try:
        response = requests.get(url, headers=get_random_headers(), timeout=15)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")

        content = soup.select_one("div.view_cnt") or soup.select_one("td.view_cnt")
        target_price_elem = soup.select_one("div.view_info_1 .money strong")
        opinion_elem = soup.select_one("div.view_info_1 .coment")

        # 정제
        target_price = (
            target_price_elem.get_text(strip=True) if target_price_elem else "N/A"
        )
        opinion = opinion_elem.get_text(strip=True) if opinion_elem else "N/A"

        clean_text = "[내용 없음]"
        if content:
            for tag in content(["script", "style", "a", "img"]):
                tag.decompose()
            clean_text = "\n".join(line.strip() for line in content.stripped_strings)

        return {"본문": clean_text, "목표가": target_price, "투자의견": opinion}

    except requests.exceptions.RequestException as e:
        print(f"[요청 실패] {url} | {type(e).__name__}: {e}")
    except Exception as e:
        print(f"[파싱 실패] {url} | {type(e).__name__}: {e}")

    return {"본문": "[크롤링 실패]", "목표가": "N/A", "투자의견": "N/A"}


def save_by_monthly_folder(df, base_folder="report_data", date_col="작성일"):
    if date_col not in df.columns:
        raise ValueError(f"'{date_col}' 컬럼이 존재하지 않습니다.")

    try:
        df[date_col] = pd.to_datetime(df[date_col], format="%y.%m.%d", errors="coerce")
    except Exception as e:
        raise ValueError(f"날짜 파싱 중 오류: {e}")

    df = df.dropna(subset=[date_col])
    if df.empty:
        print("⚠️ 유효한 날짜 데이터가 없습니다.")
        return

    for (y, m), g in df.groupby([df[date_col].dt.year, df[date_col].dt.month]):
        path = os.path.join(base_folder, f"{y}/{m:02}")
        os.makedirs(path, exist_ok=True)
        file_path = os.path.join(path, f"report_{y}_{m:02}.csv")
        g.to_csv(file_path, index=False, encoding="utf-8-sig")


def crawl_all(df, base_folder="report_data"):
    print("🟢 크롤링 시작")
    start_time = time.time()

    df["본문"] = ""
    df["목표가"] = ""
    df["투자의견"] = ""

    for idx, row in df.iterrows():
        result = get_report_content(row["URL"])
        df.at[idx, "본문"] = result["본문"]
        df.at[idx, "목표가"] = result["목표가"]
        df.at[idx, "투자의견"] = result["투자의견"]

        time.sleep(1.5)

        if (idx + 1) % 10 == 0:
            elapsed = time.time() - start_time
            print(f"📦 {idx + 1}/{len(df)}개 처리 완료 | 경과: {elapsed:.1f}초")

    # ✅ 월별 저장 함수 호출
    save_by_monthly_folder(df, base_folder)

    print("\n✅ 처리 완료:")
    print(f"- 총 항목 수: {len(df)}")
    print(f"- 본문 수집 성공률: {(df['본문'] != '[크롤링 실패]').sum() / len(df):.1%}")

In [ ]:
crawl_all(df)

🟢 크롤링 시작
📦 10/15개 처리 완료 | 경과: 15.8초

✅ 처리 완료:
- 총 항목 수: 15
- 본문 수집 성공률: 100.0%
